In [ ]:
%matplotlib inline
import sys
sys.path.append(r'D:/measuring')
sys.path.append(r'D:/analysis')
import numpy as np
import analysis.lib.QEC.hyperfine_params as hf
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import analysis.lib.nv.NV_generator as NV_generator ; reload(NV_generator)
import matplotlib as matplotlib



In [ ]:
''''''
Ap, Ao ,r, x,y,z= NV_generator.Generate_NV(Carb_Conc=0.011,N_NV=1,N=25,do_sphere = True) 
omega_l=Ap

for ii in range (0,len(Ap)):
    if abs(Ap[ii])> 10e3:
        Ap[ii]=0
        Ao[ii]=0
        
    if abs(Ao[ii])> 10e3:
        Ap[ii]=0
        Ao[ii]=0
    omega=omega_larmor_o*(0.0000001*np.random.randn()+1) 
    omega=(round(omega,8)) # convert float into string
    omega_l[ii]=omega
    

# print Ap
# print Ao
# print omega_l

In [ ]:
    
    #Ap, Ao ,r, x,y,z= NV_generator.Generate_NV(Carb_Conc=0.011,N_NV=1,N=35,do_sphere = True) 

#     inputs
#     ------
#     HFs_par:        list of parallel hyperfine components in RadHz
#     HFs_orth:       list of orthogonal hyperfine components in RadHz
#     B_field:        Magnetic field in Gauss
#     N:              number of pulses
#     tau:            time in s
#     -------
#     returns
#     -------
    
#     M:       list of signals of individual simulated spins
#     measured signal is M.prod(axis=0)

#     A_par_list=[-6.283*21.2e3]#,6.283*36.0e3,-6.283*24.7e3,6.283*12.0e3,6.283*48.7e3,-6.283*14.5e3,-6.283*7.32e3,-6.283*8.1e3]
#     A_per_list=[6.283*43.0e3]#,6.283*25.0e3,6.283*26.0e3,6.283*50.0e3,6.283*12.0e3,6.283*11.0e3,6.283*5.0e3,6.283*21.0e3]
    
   
    
    A_par_list=[-6.283*20.9e3,6.283*36.4e3,-6.283*24.65e3,6.283*12.5e3,6.283*48.55e3,-6.283*14.5e3,-6.283*7.32e3,-6.283*8.1e3]
    A_per_list=[6.283*43.0e3,6.283*25.0e3,6.283*26.0e3,6.283*50.0e3,6.283*12.0e3,6.283*11.0e3,6.283*5.0e3,6.283*21.0e3]
    
#     A_par_list=[-6.283*8.1e3]
#     A_per_list=[6.283*21.0e3]
    
    # 48.6&13 not will fitted yet
    
#     A_par_list.extend(6.283*Ap) 
#     A_per_list.extend(6.283*Ao)
    
    print A_par_list
    
    B_field=403.553*1.0
    
    N=32
    show_plot = True
################################################################################################
    
    gamma_c = 1.0705e3                           ### g-factor for C13 in Hz/G
    omega_larmor_o = 2*np.pi*gamma_c*B_field      ### Radial frequency
    tau_larmor = 2*np.pi/omega_larmor_o           ### Larmor period in seconds
    print 'tau larmor is  '+ str(tau_larmor)
    print '0.001 tau larmor is  '+ str(0.001*tau_larmor)
    omega_larmor=[omega_larmor_o,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000,omega_larmor_o*1.000]
    #omega_larmor.extend(omega_l)
    #omega_larmor=omega_larmor
    ##########
    
    
    tau=np.linspace(1*tau_larmor,120*tau_larmor,100000)
    tau=tau+0e-9
    tau=list(tau)
#     for i in range (0,len(tau)):
#         tau[i]=round(tau[i]*0.25e9)/(0.25e9) 
    tau=np.array(tau)
    
    if len(A_par_list) != len(A_per_list):
        print 'Error: Hyperfine lists lengths not equal'
        

    M=np.zeros([len(A_par_list),len(tau)])

    for kk,HF_par in enumerate(A_par_list):
        HF_orth = A_per_list[kk]

        ### equations based on Taminiau PRL 2012
        omega_tilde = np.sqrt((HF_par+omega_larmor[kk])**2+HF_orth**2)
        alpha       = omega_tilde*tau
        beta        = omega_larmor[kk]*tau

        mx          = HF_orth/omega_tilde
        mz          = (HF_par+omega_larmor[kk])/omega_tilde
        vec_term    = mx**2 *((1-np.cos(alpha))*(1-np.cos(beta)))/(1+np.cos(alpha)*np.cos(beta)-mz*np.sin(alpha)*np.sin(beta))
        angle_term  = np.sin(N*np.arccos(np.cos(alpha)*np.cos(beta)-mz*np.sin(alpha)*np.sin(beta))/2)**2

        M[kk,:]= 1-(vec_term*angle_term)

    ### get final results by multiplying the individual results
    Signal      = M.prod(axis=0)
    Fidelity    = ((Signal+1)/2)

    ### plotting
    if show_plot == True:

        plt.figure(1)
        colors = cm.rainbow(np.linspace(0, 1, len(M[:,1])))
        for kk in range(len(M[:,1])):
            plt.plot(tau*1e3, M[kk][:], '-o', lw=1, label = 'spin' + str(kk+1), color = colors[kk])

        plt.title('Signal'); plt.xlabel('Tau')
        plt.legend(loc = 4)
        plt.figure(figsize=(26,8))
        plt.ylim(0.93,1.01)
        plt.grid()



        plt.figure(2)
        plt.plot(tau*1e3, Signal, 'o-',lw=1)
        plt.title('Signal'); plt.xlabel('Tau (ms)')
        plt.ylim(-0.2,1.01)
        plt.figure(figsize=(160,8))
        plt.grid()
        
        
        
        fig=plt.figure(3)
        ax=fig.add_subplot(111)
        
        #plt.figure(3)
        ax.plot(tau*1.e6, Fidelity, '-',lw=1)
        #ax.xaxis.set_ticks(range(24,32))
        plt.title('Fidelity'); plt.xlabel('Tau (ms)')
        plt.ylim(0.0,1.01)
        plt.xlabel('Tau')
        plt.figure(figsize=(160,8))
        
        
        
        
        
    print Fidelity
    #print M


In [ ]:
'''Script to analyze the dynamical decoupling data
by THT''' 

import numpy as np
import os
from analysis.lib.tools import toolbox
from analysis.lib.tools import plot
from analysis.lib.fitting import fit, common
from analysis.lib.m2.ssro import mbi
from matplotlib import pyplot as plt

from scipy.stats.stats import pearsonr
reload(common)

'''
### Inputs ###

## Data location ##
measurement_name = ['adwindata']
timestamp = []
temperature_list = []
absolute_time_list = []


new_tsmp = '20170124_145900' ## newer than
old_tsmp = '20170130_160300' ## older than    For N=32 sweep tau

# new_tsmp = '20170301_225100' ## newer than
# old_tsmp = '20170302_180500' ## older than   For N=64  sweep tau 4ns resolution

# new_tsmp = '20170302_182700' ## newer than
# old_tsmp = '20170303_170000' ## older than   For N=256  sweep tau 4ns resolution

# new_tsmp = '20170303_172200' ## newer than
# old_tsmp = '20170303_181200' ## older than   For N=8  sweep tau 12ns resolution

# new_tsmp = '20170303_181200' ## newer than
# old_tsmp = '20170303_192000' ## older than   For N=16  sweep tau 12ns resolution



search_string = 'DecouplingSequence_111_1_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)

    absolute_time = int(old_tsmp[-2:]) + int(old_tsmp[-4:-2])*60 + int(old_tsmp[-6:-4])*60**2 + int(old_tsmp[6:8])*(60**2)*24
    absolute_time_list.append(absolute_time)
    timestamp.append(old_tsmp)


timestamp = timestamp[::-1]
absolute_time_list = absolute_time_list[::-1]

print 'time stamp is    ' +str(timestamp)

offset      = 0.5
amplitude   = 0.45
position    = 0
T2          = 150
power       = 1.5

## other settings ##
plot_fit    = False
show_guess  = False

fit_results = []

cum_pts = 0
cum_sweep_pts       = np.empty(0)
cum_p0              = np.empty(0)
cu                  =np.empty(0) 
cu_u                =np.empty(0) 
cum_u_p0            = np.empty(0)
cum_tau_list            = np.empty(0)

cum_normalized_ssro = np.empty(0)
#for k in range(0,len(measurement_name)):

for kk in range(len(timestamp)):
    folder = toolbox.data_from_time(timestamp[kk])

    a = mbi.MBIAnalysis(folder)
    a.get_sweep_pts()
    a.get_readout_results(name='adwindata')
    a.get_electron_ROC()
    cum_pts += a.pts
    # temperature = (a.g.attrs['temp']-100)/0.385
    # temperature_list.append(temperature)


    if kk == 0:
        cum_sweep_pts = a.sweep_pts
        cum_p0 = a.p0
        cum_u_p0 = a.u_p0
        #cum_tau_list = a.tau_list
    #elif kk in [5,10,15,21,26,31]: 
    else:
        cum_sweep_pts = np.concatenate((cum_sweep_pts, a.sweep_pts))
        cum_p0 = np.concatenate((cum_p0, a.p0))
        cum_u_p0 = np.concatenate((cum_u_p0, a.u_p0))
        #cum_tau_list = np.concatenate((cum_tau_list, a.tau_list))

#print cum_sweep_pts
#print cum_p0
#print cum_u_p0
'''
pow=-1.0*np.power(1.0e6*tau/(275),2.4)
decay = 0.5+0.5*np.power(2.71828,pow)
decay = 1.*np.power(2.71828,pow)
ticks=range(10,250,10)

plt.figure(1)
plt.figure(figsize=(40,10))
colors = cm.rainbow(np.linspace(0, 1, len(M[:,1])))

#for kk in range(len(M[:,1])):
#    plt.plot(tau*1e6, (1+M[kk][:]*decay)/2, '-o', lw=1, label = 'spin' + str(kk+1), color = colors[kk])
plt.plot(1e6*tau, ((Signal*decay+1)/2), '-',color='c',lw=2)
plt.plot(cum_sweep_pts,cum_p0, 'o',color='b',lw=2)
#plt.legend('Exp','Sim')
# plt.axvline(x=2.314, ymin=0, ymax=1, linewidth=0.5)
# plt.axvline(x=2*2.314, ymin=0, ymax=1, linewidth=0.5)
#plt.text(2.314, 0.5, 'tau_larmor')
# plt.errorbar(1e6*tau, Fidelity,fmt='-o',color='r',lw=4)
# plt.errorbar(1e6*tau, Fidelity,fmt='-o',color='r',lw=4)
plt.title('Fidelity')#; plt.xlabel('Tau (ms)')



plt.xticks(ticks)
plt.xlim(10,250)
#plt.ylim(0.4,1.01)
plt.xlabel('Tau (us)')
plt.grid()

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)

print folder
plt.savefig(os.path.join(folder, 'Sweep_tau_N_32_with_sim_2.pdf'),
format='pdf')
plt.savefig(os.path.join(folder, 'Sweep_tau_N_32_with_sim.png'),
format='png')


In [ ]:

'''Script to analyze the dynamical decoupling data for eigenstate
by MA''' 

import numpy as np
import os
from analysis.lib.tools import toolbox
from analysis.lib.tools import plot
from analysis.lib.fitting import fit, common
from analysis.lib.m2.ssro import mbi
from matplotlib import pyplot as plt

from scipy.stats.stats import pearsonr
reload(common)


### Inputs ###

## Data location ##
measurement_name = ['adwindata']
timestamp1 = []
timestamp2 = []
timestamp3 = []
timestamp4 = []
timestamp5 = []
timestamp6 = []
timestamp7 = []
timestamp8 = []
timestamp9 = []
timestamp10 = []
timestamp11 = []
timestamp12 = []
timestamp13 = []
timestamp14 = []
timestamp15 = []
timestamp16 = []
timestamp17 = []
timestamp18 = []
timestamp19 = []
timestamp20 = []
timestamp21 = []

timestamp64 = []
timestamp128=[]
timestamp256=[]
timestamp512=[]
timestamp1024=[]
timestamp16_m7=[]
timestamp16_p4=[]
timestamp16_m2=[]
temperature_list = []
absolute_time_list = []

# ##############################################
# new_tsmp = '20161222_233800' ## newer than
# old_tsmp = '20161223_010000' ## older than
# search_string = 'DecouplingSequence_111_1_sil18_RepT_ShutterNO_XY8sweep_tau_N_64'
# while toolbox.latest_data(contains=search_string,
#                                         return_timestamp =True,
#                                         older_than=old_tsmp,
#                                         newer_than=new_tsmp,
#                                         raise_exc=False) != False:
#     old_tsmp, folder = toolbox.latest_data(contains=search_string,
#                                         return_timestamp =True,
#                                         older_than=old_tsmp,
#                                         newer_than=new_tsmp,
#                                         raise_exc=False)


#     timestamp64.append(old_tsmp)

# timestamp64 = timestamp64[::-1]
# # # print timestamp64


#####################################################
new_tsmp = '20170211_233100' ## newer than
old_tsmp = '20170212_002000' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp1.append(old_tsmp)
    
timestamp1 = timestamp1[::-1]
print timestamp1


#########################################################
new_tsmp = '20170212_002500' ## newer than
old_tsmp = '20170212_012000' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp2.append(old_tsmp)
    
timestamp2 = timestamp2[::-1]

# ###########################################
new_tsmp = '20170212_012200' ## newer than
old_tsmp = '20170212_021200' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp3.append(old_tsmp)
    
timestamp3 = timestamp3[::-1]

# #################################

new_tsmp = '20170212_021500' ## newer than
old_tsmp = '20170212_030800' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp4.append(old_tsmp)
    
timestamp4 = timestamp4[::-1]

# #################################

new_tsmp = '20170212_031500' ## newer than
old_tsmp = '20170212_040700' ## older than

search_string = '_DecouplingSequence_111_1_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp5.append(old_tsmp)
    
timestamp5 = timestamp5[::-1]

# #################################

new_tsmp = '20170212_041000' ## newer than
old_tsmp = '20170212_050400' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp6.append(old_tsmp)
    
timestamp6 = timestamp6[::-1]

# #################################

new_tsmp = '20170212_050500' ## newer than
old_tsmp = '20170212_060200' ## older than

search_string = '_DecouplingSequence_111_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp7.append(old_tsmp)
    
timestamp7 = timestamp7[::-1]
#################################

# #################################

new_tsmp = '20170212_060200' ## newer than
old_tsmp = '20170212_070000' ## older than

search_string = '_DecouplingSequence_111'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp8.append(old_tsmp)
    
timestamp8 = timestamp8[::-1]

# #################################

new_tsmp = '20170212_070000' ## newer than
old_tsmp = '20170212_075400' ## older than

search_string = '_DecouplingSequence_111_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp9.append(old_tsmp)
    
timestamp9 = timestamp9[::-1]

# #################################

new_tsmp = '20170212_075500' ## newer than
old_tsmp = '20170212_084600' ## older than

search_string = 'DecouplingSequence_111_1_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp10.append(old_tsmp)
    
timestamp10 = timestamp10[::-1]

# #################################

new_tsmp = '20170212_085200' ## newer than
old_tsmp = '20170212_094500' ## older than

search_string = 'DecouplingSequence_111_1_'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp11.append(old_tsmp)
    
timestamp11 = timestamp11[::-1]

# #################################

new_tsmp = '20170211_181700' ## newer than
old_tsmp = '20170211_200000' ## older than

search_string = '_DecouplingSequence_111'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp12.append(old_tsmp)
    
timestamp12 = timestamp12[::-1]

# #################################

new_tsmp = '20170204_181900' ## newer than
old_tsmp = '20170204_200000' ## older than

search_string = '_DecouplingSequence_111_1_sil18_RepT'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp13.append(old_tsmp)
    
timestamp13 = timestamp13[::-1]

# #################################

new_tsmp = '20170204_201300' ## newer than
old_tsmp = '20170204_233900' ## older than

search_string = '_DecouplingSequence_111_1_sil18_S'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp14.append(old_tsmp)
    
timestamp14 = timestamp14[::-1]

# #################################

new_tsmp = '20170204_201300' ## newer than
old_tsmp = '20170204_233900' ## older than

search_string = '_DecouplingSequence_111_1_sil18_RepT'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp15.append(old_tsmp)
    
timestamp15 = timestamp15[::-1]

# #################################

new_tsmp = '20170204_035400' ## newer than
old_tsmp = '20170204_101100' ## older than

search_string = '_DecouplingSequence_111_1_sil18_S'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp16.append(old_tsmp)
    
timestamp16 = timestamp16[::-1]

# #################################

new_tsmp = '20170204_035400' ## newer than
old_tsmp = '20170204_100000' ## older than

search_string = '_DecouplingSequence_111_1'
while toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False) != False:
    old_tsmp, folder = toolbox.latest_data(contains=search_string,
                                        return_timestamp =True,
                                        older_than=old_tsmp,
                                        newer_than=new_tsmp,
                                        raise_exc=False)


    timestamp17.append(old_tsmp)
    
timestamp17 = timestamp17[::-1]


timestamp_list=[timestamp1,timestamp2,timestamp3,timestamp4,timestamp5,timestamp6,timestamp7,timestamp8,timestamp9,timestamp10,timestamp11]#,timestamp12,timestamp13,timestamp14,timestamp15,timestamp16,timestamp17]
labels = ['4.60','4.61','4.62','4.63','4.64','4.65','4.66','4.67','4.68','4.69','4.70','4.4']#['64-Eigenstate-optimal','64-Eigenstate-overrotation','64-Superposition-optimal','256-Eigenstate-optimal','256-Eigenstate-over','256-superposition','-0.8','-0.6','-0.4','-0.2','0','+0.2','+0.4','0.6','+0.8','+1','1.2','1.4','1.6']
N = [3072,3072,3072,3072,3072,3072,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,]
color_list =  ['b','r','g','c','y','k','m','b','r','g','c','y','k','m','b','r','g','c','y','k','m']
fmt_label = ['-o','-o','-o','-o','-o','-o','-o','--o','--o','--o','--o','--o','--o','--o','--o']


print 'time stamp is    ' +str(timestamp_list)

offset      = 0.5
amplitude   = 0.45
position    = 0
T2          = 150
power       = 1.5

## other settings ##
plot_fit    = True
show_guess  = False

fit_results = []

cum_pts = 0
cum_sweep_pts       = np.empty(0)
cum_p0              = np.empty(0)
cu                  =np.empty(0) 
cu_u                =np.empty(0) 
cum_u_p0            = np.empty(0)

cum_normalized_ssro = np.empty(0)
#for k in range(0,len(measurement_name)):

for ii, timestamp in enumerate(timestamp_list):
    print timestamp

    for kk in range(len(timestamp)):

        folder = toolbox.data_from_time(timestamp[kk])

        a = mbi.MBIAnalysis(folder)
        a.get_sweep_pts()
        a.get_readout_results(name='adwindata')
        a.get_electron_ROC()
        cum_pts += a.pts
        temperature = (a.g.attrs['temp']-100)/0.385
        temperature_list.append(temperature)


        if kk == 0:
            cum_sweep_pts = a.sweep_pts
            cum_p0 = a.p0
            cum_u_p0 = a.u_p0
        else:
            cum_sweep_pts = np.concatenate((cum_sweep_pts, a.sweep_pts))
            cum_p0 = np.concatenate((cum_p0, a.p0))
            cum_u_p0 = np.concatenate((cum_u_p0, a.u_p0))

    #print cum_sweep_pts
    #print cum_p0
    #print cum_u_p0
    #relative_time   = (np.array(absolute_time_list)-absolute_time_list[0])/60.


    a.pts   = cum_pts
    a.sweep_pts = cum_sweep_pts
    a.p0    = cum_p0
    a.u_p0  = 0*cum_u_p0


    # if ii == 0:
    #     ax = a.plot_results_vs_sweepparam(ret='ax',labels=[labels[ii]],figsize=(10,6))
    #     x_max = max(cum_sweep_pts)
    #     x_min = min(cum_sweep_pts)    
    # else:
    #     cum_sweep_pts = np.array(cum_sweep_pts)
    #     cum_p0 = np.array(cum_p0)
    #     # cum_u_p0 = np.array(cum_u_p0)
    #     x_max = max(x_max,max(cum_sweep_pts))
    #     x_min = min(x_min,min(cum_sweep_pts))
    #     cum_u_p0_ar = cum_u_p0
    #     cum_u_p0_ar = np.reshape(cum_u_p0_ar, (np.shape(cum_u_p0_ar)[0], ))
    #     # print np.shape(cum_u_p0)
    #     plt.errorbar(cum_sweep_pts.flatten(),cum_p0.flatten(),yerr=cum_u_p0_ar,fmt='o',label=labels[ii],color=color_list[ii])

   

    # x = a.sweep_pts.reshape(-1)[:]
    # y = a.p0.reshape(-1)[:]

    # p0, fitfunc, fitfunc_str = common.fit_general_exponential(offset, amplitude, position, T2, power)

    # # plt.plot(cum_sweep_pts,cum_p0,'o')

    # #plot the initial guess
    # # if show_guess:
    # #     ax.plot(np.linspace(0,x[-1],201), fitfunc(np.linspace(0,x[-1],201)), '-', lw=2)

    # fit_result = fit.fit1d(x,y, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=[0,2])

    # ## plot data and fit as function of total time
     
    # if plot_fit == True:
    #     plot.plot_fit1d(fit_result, np.linspace(0,np.amax(x)+10.,201), ax=ax,add_txt = False, plot_data=False,color=color_list[ii])

    # fit_results.append(fit_result)
    # cum_pts = 0
    # cum_sweep_pts       = np.empty(0)
    # cum_p0              = np.empty(0)
    # cum_u_p0              = np.empty(0)



###################################################### plot against tau (in tau larmor)

    a.sweep_pts = 1000*cum_sweep_pts/(2*8192)

    if ii == 0:
        ax = a.plot_results_vs_sweepparam(ret='ax',name='fidelity_vs_tau',fmt='-o',labels=[labels[ii]],figsize=(40,8))
        x_max = max(1000*cum_sweep_pts/(2*2.3158*N[ii]**1))
        x_min = min(1000*cum_sweep_pts/(2*2.3158*N[ii]**1))    
    else:
        cum_sweep_pts = np.array(cum_sweep_pts)
        cum_p0 = np.array(cum_p0)
        # cum_u_p0 = np.array(cum_u_p0)
        x_max = max(x_max,max(1000*cum_sweep_pts/(2*2.3158*N[ii]**1.)))
        x_min = min(x_min,min(1000*cum_sweep_pts/(2*2.3158*N[ii]**1.)))
        cum_u_p0_ar = cum_u_p0
        cum_u_p0_ar = np.reshape(cum_u_p0_ar, (np.shape(cum_u_p0_ar)[0], ))
        # print np.shape(cum_u_p0)
        plt.errorbar(cum_sweep_pts.flatten()/(1e-3*2*8192),cum_p0.flatten(),yerr=0*cum_u_p0_ar,fmt=fmt_label[ii],label=labels[ii],color=color_list[ii])

    # x = a.sweep_pts.reshape(-1)[:]
    # y = a.p0.reshape(-1)[:]

    # p0, fitfunc, fitfunc_str = common.fit_general_exponential(offset, amplitude, position, T2, power)

  

    # # plot the initial guess
    # # if show_guess:
    # #     ax.plot(np.linspace(0,x[-1],201), fitfunc(np.linspace(0,x[-1],201)), '-', lw=2)

    # # fit_result = fit.fit1d(x,y, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=[0,2])

    # ## plot data and fit as function of total time
     
    # # if plot_fit == True:
    # #     plot.plot_fit1d(fit_result, np.linspace(0,np.amax(x)+10.,201), ax=ax,add_txt = False, plot_data=False,color=color_list[ii])

    # # fit_results.append(fit_result)
    cum_pts = 0
    cum_sweep_pts       = np.empty(0)
    cum_p0              = np.empty(0)
    cum_u_p0              = np.empty(0)
#############################################################################################

#plt.errorbar(1e6*tau, Fidelity,yerr=0*tau,fmt='-o',label='Simulation',color='b')

print folder
ax.set_ylim(0.3,1.02)
# ax.set_xlim(0.,x_max)
ax.set_xlabel('Tau (us) ')
plt.axvline(x=2.314, ymin=0, ymax=1, linewidth=0.5)
plt.grid()
ax.set_xlim(2.2,2.42)
# ax.set_xscale('log')
plt.legend()
plt.savefig(os.path.join(folder, 'sweep_Tau_align_field_with_sim.pdf'),
format='pdf')
plt.savefig(os.path.join(folder, 'combined_result.png'),
format='png')





In [ ]:
decay = np.power(2.71828,-1.0e6*tau/180)
decay = 0.488+np.power(decay,2.2)
print decay

In [ ]:
# decay = np.power(2.71828,-1.0e6*tau/124.7719)
# decay = 0.513+0.5*np.power(decay,1.7)


pow=-1.0*np.power(1.0e6*tau/124.7719,1.7)
decay = 0.513+0.5*np.power(2.71828,pow)
plt.plot(1.0e6*tau,decay)

In [ ]:
print np.power([1,2,3],2)

In [ ]:
reload(NV_generator)
NV_generator.Generate_NV(Carb_Conc=0.011,N_NV=1,N=25,do_sphere = True)

In [ ]:
1000*cum_sweep_pts/(2*32)

In [ ]:
ticks=range(10,250,10)
print ticks

In [ ]:
(1+M[3][:])/2

In [ ]:
np.save('sweep_tau_32_cum_P',cum_p0)

In [ ]:
np.save('sweep_tau_32_cum_sweep_pts',cum_sweep_pts)

In [ ]:
a=np.load('sweep_tau_32_cum_P')